In [1]:
import numpy as np

#modified on 6/23/2021
#setup a in main.py with shape changed from (nelem,) to (nnode,)
#setup a in main.py with shape changed from (nelem,) to (nnode,)
#replaced perm with rel_perm
#changed cinf unit from m^-3 to mol*m^-3 by removing kA from its value

mesh_prefix='sphere'
outfile='sphere_s22.npz'

#set physical constants and properties
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kB=1.381e-23 #Boltzmann's constant [J/K]
kA=6.022e23 #Avogadro constant [1/mol]

rel_perm=80 #relative permittivity of bulk electrolyte [F/m]
tK=293 #room/ambient temperature [K]
cinf=1 #ion concentration in bulk electrolyte [mol/m^3]
zval=1 #ion valence in bulk electrolyte
# cinf_pos=0.01 #ion concentration of positive charges
# cinf_neg=0.01 #concentration of negative charges
# zval_pos=1 #positive ion valence
# zval_neg=1 #negative ion valence

rel_perm=78.5
tK=298
cinf=100
zval=1

#load mesh
print('Reading %s.1.node'%mesh_prefix)
nodes=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=(1,2,3))
node_flags=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=5,dtype='int')

print('Reading %s.1.ele'%mesh_prefix)
elements=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=(1,2,3,4),dtype='int')
zones=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=5,dtype='int')

print('Reading %s.1.face'%mesh_prefix)
faces=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=(1,2,3),dtype='int')
face_flags=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=4,dtype='int')

#adjust indices to start from zero
elements=elements-1
faces=faces-1

# #translate z coordinate
# print('Translating Z coordinate')
# nodes[:,2]=-(np.power(10,-nodes[:,2]/1000*3)-1) #problem dependent
dist1=np.sqrt(nodes[:,0]**2+nodes[:,1]**2+nodes[:,2]**2)
mask=dist1>5.0
dist2=np.zeros_like(dist1)
dist2[mask]=10**((dist1[mask]-5)/50*2)+5-1

print(min(dist1[mask]),max(dist1[mask]))
print(min(dist2[mask]),max(dist2[mask]))
nodes[mask,0]=nodes[mask,0]/dist1[mask]*dist2[mask]
nodes[mask,1]=nodes[mask,1]/dist1[mask]*dist2[mask]
nodes[mask,2]=nodes[mask,2]/dist1[mask]*dist2[mask]

#scale nodes from meter to nano-meter
nodes=nodes*1e-9

nnode=len(nodes)
nelem=len(elements)
nface=len(faces)
print('THE NUMBER OF NODES IS: %d'%nnode)
print('THE NUMBER OF ELEMENTS IS: %d'%nelem)
print('THE NUMBER OF FACES IS: %d'%nface)

#set boolean indicies for volume elements, S1 nodes, and S2 elements
mask_e=zones<=4 #all elements
# mask_d=node_flags==1 #boolean node indices True for Dirichlet nodes
# mask_s=face_flags==1 #boolean face indices True for third kind surface elements
mask_d=(node_flags==1)|(node_flags==2) #outer boundary may need to be updated
#mask_s=np.zeros_like(face_flags) #empty for this case
mask_s=face_flags==20 #interface between zone 1 and 2

#initialize scalar PDE coefficients
print('Setting PDE coefficients')
cx=rel_perm*np.ones(nelem) #cx scaled by perm0
cy=rel_perm*np.ones(nelem) #cy scaled by perm0
cz=rel_perm*np.ones(nelem) #cz scaled by perm0

cx[(zones==1)|(zones==2)]=4.5
cy[(zones==1)|(zones==2)]=4.5
cz[(zones==1)|(zones==2)]=4.5

alphax=np.zeros(nelem) #unused so far
alphay=np.zeros(nelem) #unused so far
alphaz=np.zeros(nelem) #unused so far

betax=np.zeros(nelem) #unused so far
betay=np.zeros(nelem) #unused so far
betaz=np.zeros(nelem) #unused so far

gammax=np.zeros(nelem) #unused so far
gammay=np.zeros(nelem) #unused so far
gammaz=np.zeros(nelem) #unused so far

a=np.zeros(nnode)
f=np.zeros(nnode)
# a='pb' #will be setup in main.py with shape (nnode,)
# f='pb' #will be setup in main.py with shape (nnode,)

g=np.zeros(nface) #length of ns for essential entries
q=np.zeros(nface) #length of ns for essential entries
s=np.zeros(nnode) #length of nd for essential entries

#set Dirichlet B.C on solid interface
#for monovalent ions (zval=1), we can use linear PB equation for phi_e<<26mV
#for divalent ions (zval=2), we can use linear PB equation for phi_e<<13 mv
#s[node_flags==10]=0.1 #phi_e=0.1V
g[face_flags==20]=0.01/perm0 #surface charge density [C/m^2] scaled by perm0

#initialize vector PDE coefficients
cx3=np.zeros((nelem,3,3)) #PDE term coefficient
cy3=np.zeros((nelem,3,3)) #PDE term coefficient
cz3=np.zeros((nelem,3,3)) #PDE term coefficient

alphax3=np.zeros((nelem,3,3)) #PDE term coefficient
alphay3=np.zeros((nelem,3,3)) #PDE term coefficient
alphaz3=np.zeros((nelem,3,3)) #PDE term coefficient

betax3=np.zeros((nelem,3,3)) #PDE term coefficient
betay3=np.zeros((nelem,3,3)) #PDE term coefficient
betaz3=np.zeros((nelem,3,3)) #PDE term coefficient

gammax3=np.zeros((nelem,3,3)) #PDE term coefficient
gammay3=np.zeros((nelem,3,3)) #PDE term coefficient
gammaz3=np.zeros((nelem,3,3)) #PDE term coefficient

a3=np.zeros((nelem,3,3)) #PDE term coefficient
f3=np.zeros((nelem,3)) #source term coefficient
g3=np.zeros((nface,3)) #Neumann boundary condition coefficient
q3=np.zeros((nface,3,3)) #Neumann boundary condition coefficient
s3=np.zeros((nnode,3)) #Dirichlet boundary condition coefficient

cx3[:,0,0]=1
cx3[:,0,2]=1
cx3[:,1,1]=1
cx3[:,1,2]=1
cx3[:,2,2]=1

cy3[:,0,0]=1
cy3[:,0,2]=1
cy3[:,1,1]=1
cy3[:,1,2]=1
cy3[:,2,2]=1

cz3[:,0,0]=1
cz3[:,0,2]=1
cz3[:,1,1]=1
cz3[:,1,2]=1
cz3[:,2,2]=1

a3[:,0,0]=1
a3[:,1,1]=1
a3[:,2,0]=1
a3[:,2,2]=1

g3[:,2]=10
s3[:,2]=1

#save PDE coefficients to a file
print('Saving PDE coefficients to %s'%outfile)
np.savez(outfile,nodes=nodes,elements=elements,faces=faces,
         node_flags=node_flags,zones=zones,face_flags=face_flags,
         mask_e=mask_e,mask_d=mask_d,mask_s=mask_s,
         cx=cx,cy=cy,cz=cz,
         alphax=alphax,alphay=alphay,alphaz=alphaz,
         betax=betax,betay=betay,betaz=betaz,
         gammax=gammax,gammay=gammay,gammaz=gammaz,
         a=a,f=f,g=g,q=q,s=s,
         cx3=cx3,cy3=cy3,cz3=cz3,
         alphax3=alphax3,alphay3=alphay3,alphaz3=alphaz3,
         betax3=betax3,betay3=betay3,betaz3=betaz3,
         gammax3=gammax3,gammay3=gammay3,gammaz3=gammaz3,
         a3=a3,f3=f3,g3=g3,q3=q3,s3=s3,
         rel_perm=rel_perm,tK=tK,cinf=cinf,zval=zval)
print('')


Reading sphere.1.node
Reading sphere.1.ele
Reading sphere.1.face
5.000058817654048 87.4685657822283
5.000005417320811 1993.4939888608171
THE NUMBER OF NODES IS: 24157
THE NUMBER OF ELEMENTS IS: 135851
THE NUMBER OF FACES IS: 17578
Setting PDE coefficients
Saving PDE coefficients to sphere_s22.npz

